In [1]:
import numpy as np
import pandas as pd

# Population Data Cleaning

In [2]:
# import data
pop_total = pd.read_csv('data-sources/population_total.csv')

In [3]:
def remove_years(df, start_year=2000):
    # remove all year columns from df that are before start_year
    
    df_copy = df.copy()
    
    # collect column headers to be removed
    cols = []
    for col in df_copy:
        if (col.isdigit() and int(col) < start_year):
            cols.append(col)

    # drop columns
    df_copy = df_copy.drop(columns=cols, axis=1)
    
    return df_copy
    
pop_total_short = remove_years(pop_total, start_year=2000)

In [4]:
# remove columns otherwise not required
pop_cols = ['Indicator Name', '2018', 'Indicator Code', 'Country Code']
pop_total_short = pop_total_short.drop(columns=pop_cols, axis=1)

In [5]:
# transpose dataframe
pop_total_short = pop_total_short.transpose()

# change column headers to country names & drop duplicate column
pop_total_short.columns = list(pop_total_short.loc['Country Name' , : ])
pop_total_short = pop_total_short.drop(['Country Name'])

In [6]:
# remove unions
unions = [
    "Central Europe and the Baltics", "Caribbean small states", "East Asia & Pacific (excluding high income)", 
    "Early-demographic dividend", "East Asia & Pacific", "Europe & Central Asia (excluding high income)", 
    "Europe & Central Asia", "Euro area", "European Union", "Fragile and conflict affected situations",
    "High income", "Heavily indebted poor countries (HIPC)", "IBRD only", "IDA & IBRD total", "IDA total",
    "IDA blend", "IDA only", "Latin America & Caribbean", "Not classified" ,"Latin America & Caribbean (excluding high income)", 
    "Least developed countries: UN classification", "Low income", "Lower middle income", "Low & middle income", 
    "Late-demographic dividend", "Middle East & North Africa", "Middle income", 
    "Middle East & North Africa (excluding high income)", "OECD members", "Other small states", 
    "Pre-demographic dividend", "Pacific island small states", "Post-demographic dividend", 
    "Sub-Saharan Africa (excluding high income)", "Sub-Saharan Africa", "Small states", 
    "East Asia & Pacific (IDA & IBRD countries)", "Europe & Central Asia (IDA & IBRD countries)", 
    "Latin America & the Caribbean (IDA & IBRD countries)", "Middle East & North Africa (IDA & IBRD countries)",
    "South Asia (IDA & IBRD)", "Sub-Saharan Africa (IDA & IBRD countries)", "South Asia", "Upper middle income", 
    "World", "Arab World", "North America"
]
pop_total_short = pop_total_short.drop(columns=unions, axis=1)

# remove countries with less than 10M inhabitants in 2000
small_countries = []
for col in pop_total_short:
    if pop_total_short[col]['2000'] <= 10000000:
        small_countries.append(col)

pop_total_short = pop_total_short.drop(columns=small_countries, axis=1)

# drop columns with more than two missing values
pop_total_short = pop_total_short.dropna(axis='columns', thresh=2)

In [7]:
# calculate population percentage change
pop_perc = pop_total_short.copy()

for col in pop_total_short:
    base_val = pop_total_short[col].iloc[0]
    pop_perc[col] = pop_perc[col]/base_val - 1
    
pop_perc.head()

,Afghanistan,Angola,Argentina,Australia,Belgium,Burkina Faso,Bangladesh,Brazil,Canada,Chile,...,Uganda,Ukraine,United States,Uzbekistan,"Venezuela, RB",Vietnam,"Yemen, Rep.",South Africa,Zambia,Zimbabwe
2000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2001,0.0434318,0.0329873,0.0111734,0.0135749,0.00344543,0.0290013,0.0191966,0.0140517,0.0101463,0.0119385,...,0.0339286,-0.0100046,0.00994656,0.0127402,0.01879,0.0106415,0.0288346,0.0143607,0.0278129,0.0117748
2002,0.0938683,0.0688359,0.0224494,0.026022,0.00795366,0.0590249,0.038147,0.0277455,0.0192495,0.0236446,...,0.0698346,-0.0197932,0.0193604,0.0252105,0.0375836,0.0208124,0.0584319,0.0283819,0.0559468,0.0227678
2003,0.147862,0.107199,0.0337834,0.0387616,0.0121822,0.0901692,0.056526,0.0410443,0.0294543,0.0351698,...,0.107555,-0.0277148,0.0281594,0.0372103,0.0563608,0.0306668,0.0888048,0.0419961,0.0845831,0.03368
2004,0.200322,0.147485,0.0450987,0.0508745,0.0165723,0.122556,0.0739182,0.0539164,0.0398216,0.0465856,...,0.146808,-0.0350629,0.037719,0.0492467,0.0750923,0.0403823,0.119853,0.055088,0.113997,0.0454303


In [8]:
# adjust index column
pop_perc = pop_perc.reset_index()
pop_perc = pop_perc.rename(columns={'index': 'Year'})

In [9]:
# add columns for bottom/top lines with distance representing total population
min_population = min(list(pop_total_short.min()))
max_population = max(list(pop_total_short.max()))

# how can we make this logarithmic??
#
#
max_thickness = 0.1

for col in pop_total_short:
    thickness = pop_total_short[col]*max_thickness/max_population
    thickness.index = range(18)
    
    # lower limit
    pop_perc[col + '_low'] = pop_perc[col] - thickness/2
    
    # upper limit
    pop_perc[col + '_up'] = pop_perc[col] + thickness/2

In [10]:
# store data
pop_perc.to_csv('pop_total.csv', index=False)

# Metadata Cleaning

In [11]:
# import data
metadata = pd.read_csv('data-sources/country_metadata.csv')

In [12]:
# remove columns otherwise not required
cols = ['Country Code', 'SpecialNotes']
metadata = metadata.drop(columns=cols, axis=1)

# rename "TableName" column
metadata.columns.values[-1] = 'Country'

# remove all rows with missing valuesp
print(metadata.shape)
metadata = metadata.dropna()

print(metadata.shape)

(263, 3)
(217, 3)


In [13]:
metadata.head()

,Region,IncomeGroup,Country
0,Latin America & Caribbean,High income,Aruba
1,South Asia,Low income,Afghanistan
2,Sub-Saharan Africa,Lower middle income,Angola
3,Europe & Central Asia,Upper middle income,Albania
4,Europe & Central Asia,High income,Andorra


In [14]:
# store data
metadata.to_csv('metadata.csv', index=False)